<a href="https://colab.research.google.com/github/ambideXtrous9/Multi-Approach-Similarity-Based-Article-Bias-Prediction-using-LSTM/blob/main/MHA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --user --quiet openpyxl

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install  --quiet torchmetrics
!pip install  --quiet pytorch_lightning==1.6

     |████████████████████████████████| 529 kB 43.6 MB/s 
     |████████████████████████████████| 582 kB 36.2 MB/s 


In [4]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import warnings
import torch
from torch.utils.data import DataLoader
from torch import nn
from torch.autograd import Variable
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning import Trainer
import openpyxl
warnings.filterwarnings(action='ignore',category=UserWarning)
warnings.filterwarnings(action='ignore',category=FutureWarning)
from torch.utils.data import TensorDataset, DataLoader
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
import tensorflow as tf
from tqdm import tqdm

In [6]:
import tensorflow_hub as hub
#download the model
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [7]:
import re

In [8]:
def clean(text):
    text = re.sub(r"http\S+", " ", text) # remove urls
    text = re.sub(r"RT ", " ", text) # remove rt
    text = re.sub(r"[^a-zA-Z\'\.\,\d\s]", " ", text) # remove special character except # @ . ,
    text = re.sub(r"[0-9]", " ", text) # remove number
    text = re.sub(r'\t', ' ', text) # remove tabs
    text = re.sub(r'\n', ' ', text) # remove line jump
    text = re.sub(r"\s+", " ", text) # remove extra white space
    text = text.strip()
    return text

In [9]:
def EmbeddingHead(df):
    X_train = []
    for r in tqdm(df.title.values):
      emb = use(r)
      review_emb = tf.reshape(emb, [-1]).numpy()
      X_train.append(review_emb)

    X_train = np.array(X_train)
    return X_train

In [10]:
def EmbeddingBody(df,col):
    X_train = []
    for r in tqdm(df.content_original.values):
      emb = use(r)
      review_emb = tf.reshape(emb, [-1]).numpy()
      X_train.append(review_emb)

    X_train = np.array(X_train)
    return X_train

In [11]:
train_path = '/content/drive/MyDrive/SMDM/Train.xlsx'

In [12]:
df_train = pd.read_excel(train_path,engine='openpyxl')

T1 = df_train['content_original'].str.split(' \n\n---\n\n').str[0]
df_train['content_original'] = T1.str.replace('-',' ').str.replace('[^\w\s]','').str.replace('\n',' ').str.lower()

T2 = df_train['title'].str.split(' \n\n---\n\n').str[0]
df_train['title'] = T2.str.replace('-',' ').str.replace('[^\w\s]','').str.replace('\n',' ').str.lower()


df_train.content_original = df_train.content_original.apply(clean)
df_train.title = df_train.title.apply(clean)


y_train = df_train['bias'].to_numpy()


stop = stopwords.words('english')
df_train['title'] = df_train['title'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))
df_train['content_original'] = df_train['content_original'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))

In [13]:
df_train['content_original']

0        president trump senate minority leader chuck s...
1        shows diverse perspectives days top stories de...
2        los angeles actress rosario dawson took stage ...
3        president donald trump said friday doesnt know...
4        washington cnn donald trump became th presiden...
                               ...                        
26585    marco rubio chuck schumer balance policy polit...
26586    obama aide predicts us allies syria samantha p...
26587    trump suggested times holds grudge paper faile...
26588    trump administration ordered hospitals bypass ...
26589    rush transcript copy may final form amy goodma...
Name: content_original, Length: 26590, dtype: object

In [14]:
df_train['content_original'].isna().sum()

0

In [15]:
df_train['title'].isna().sum()

0

In [16]:
df_train['bias'].value_counts()

2    10241
0     8861
1     7488
Name: bias, dtype: int64

In [17]:
import re

In [18]:
X_head = []
for r in tqdm(df_train.title.values):
  r = [r]
  emb = use(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_head.append(review_emb)

X_head = np.array(X_head)

100%|██████████| 26590/26590 [02:44<00:00, 161.24it/s]


In [19]:
X_head.shape

(26590, 512)

In [20]:
X_body = []
for r in tqdm(df_train.content_original.values):
  r = [r]
  emb = use(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_body.append(review_emb)

X_body = np.array(X_body)

100%|██████████| 26590/26590 [03:25<00:00, 129.64it/s]


In [21]:
X_body.shape

(26590, 512)

In [22]:
idx = np.random.randint(0,X_head.shape[0], size = int(0.15*X_head.shape[0]), dtype=int)
idx

array([14299, 25080,  1668, ...,  1523,  4149, 25857])

In [23]:
idx1 = np.array([i for i in range(X_head.shape[0]) if i not in idx])
idx1

array([    0,     2,     3, ..., 26586, 26588, 26589])

In [24]:
len(idx)

3988

In [25]:
len(idx1)

22884

In [26]:
X_head_val = X_head[idx]
X_body_val = X_body[idx]
y_val = y_train[idx]

X_head_train = X_head[idx1]
X_body_train = X_body[idx1]
y_train = y_train[idx1]

In [27]:
np.unique(y_train, return_counts=True)

(array([0, 1, 2]), array([7659, 6423, 8802]))

In [28]:
np.unique(y_val, return_counts=True)

(array([0, 1, 2]), array([1288, 1143, 1557]))

In [29]:
print(X_head_train.shape)
print(X_body_train.shape)
print(y_train.shape)
print(X_head_val.shape)
print(X_body_val.shape)
print(y_val.shape)

(22884, 512)
(22884, 512)
(22884,)
(3988, 512)
(3988, 512)
(3988,)


In [30]:
from numpy import newaxis

In [31]:
X_train_head_reshaped = X_head_train[:, newaxis,:]
X_train_body_reshaped = X_body_train[:, newaxis,:]
X_val_head_reshaped = X_head_val[:, newaxis,:]
X_val_body_reshaped = X_body_val[:, newaxis,:]

In [32]:
BS = 512

In [33]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [34]:
train_head_features = torch.Tensor(X_train_head_reshaped)
train_body_features = torch.Tensor(X_train_body_reshaped)
train_targets = torch.Tensor(y_train).type(torch.LongTensor)
val_head_features = torch.Tensor(X_val_head_reshaped)
val_body_features = torch.Tensor(X_val_body_reshaped)
val_targets = torch.Tensor(y_val).type(torch.LongTensor)


trainDataset = TensorDataset(train_head_features, train_body_features, train_targets)
valDataset = TensorDataset(val_head_features, val_body_features, val_targets)

In [35]:
class LitLSTM(pl.LightningModule):
    def __init__(self,num_classes,dimension,hidd_dim):
        super(LitLSTM, self).__init__()
        
        self.hidd_dim = hidd_dim
        self.n_mha = 4
        self.num_layer = 2
        self.embed_dim = hidd_dim
        self.mha = nn.MultiheadAttention(self.embed_dim,self.n_mha,batch_first=True,kdim=self.hidd_dim,vdim=hidd_dim)
        self.drop = nn.Dropout(p=0)
        self.fc = nn.Linear(self.embed_dim, 3)
        self.act = nn.Softmax()
        
        self.LSTM = nn.LSTM(input_size=dimension,
                            hidden_size=self.hidd_dim,
                            num_layers=self.num_layer,
                            batch_first=True,
                            bidirectional=True)
        
        self.model = nn.Sequential( nn.Dropout(p=0.4),
                                    nn.ReLU(),
                                    nn.Linear(self.hidd_dim, 128),
                                    nn.Dropout(p=0.4),
                                    nn.ReLU(),
                                    nn.Linear(128, num_classes))
        
 # add metrics
        self.train_acc = torchmetrics.Accuracy(num_classes=num_classes)
        self.train_f1 = torchmetrics.F1Score(num_classes=num_classes)
        self.val_acc = torchmetrics.Accuracy(num_classes=num_classes)
        self.val_f1 = torchmetrics.F1Score(num_classes=num_classes)

    def forward(self, head, body):
        h_0_head = Variable(torch.rand(2*self.num_layer, BS, self.hidd_dim)).to(device)
        c_0_head = Variable(torch.rand(2*self.num_layer, BS, self.hidd_dim)).to(device)
        h_0_body = Variable(torch.rand(2*self.num_layer, BS, self.hidd_dim)).to(device)
        c_0_body = Variable(torch.rand(2*self.num_layer, BS, self.hidd_dim)).to(device)

        head, (final_hidden_state_head, final_cell_state_head) = self.LSTM(head, (h_0_head, c_0_head))
        body, (final_hidden_state_body, final_cell_state_body) = self.LSTM(body, (h_0_body, c_0_body))
        
        h = final_hidden_state_head[-1].reshape(BS,1,-1)
        b = final_hidden_state_body[-1].reshape(BS,1,-1)
        
        mha, mha_wgts = self.mha(h,b,b)
        x = self.model(mha).reshape(-1,3)
        return x

    def training_step(self, batch, batch_idx):
        head, body, labels = batch
        # Forward pass
        outputs = self.forward(head, body)
        lossfn = nn.CrossEntropyLoss()
        loss = lossfn(outputs, labels)
        
        y_pred = torch.argmax(outputs,dim=1)
        acc = self.train_acc(y_pred, labels)
        f1 = self.train_f1(y_pred, labels)
        # just accumulate

        self.log("train_loss", loss)
        self.log("train_accuracy", acc)
        self.log("train_f1", f1)
        tensorboard_logs = {'train_loss': loss}
        # use key 'log'
        return {"loss": loss, 'log': tensorboard_logs}

    # define what happens for testing here

    def train_dataloader(self):
        trainDataLoader = DataLoader(trainDataset, num_workers=2,batch_size=BS, shuffle=True,drop_last=True)

        return trainDataLoader

    def val_dataloader(self):
        valDataLoader = DataLoader(valDataset, num_workers=2,batch_size=BS,shuffle=False,drop_last=True)
        
        return valDataLoader
    
    def validation_step(self, batch, batch_idx):
        head, body, labels = batch
        # Forward pass
        outputs = self.forward(head, body)
        lossfn = nn.CrossEntropyLoss()
        loss = lossfn(outputs, labels)
        
        pred = torch.argmax(outputs,dim=1)
        self.val_acc.update(pred, labels)
        self.val_f1.update(pred, labels)

        self.log("val_loss", loss)
        return {"val_loss": loss}
    
    def training_epoch_end(self, training_step_outputs):
        # compute metrics
        train_accuracy = self.train_acc.compute()
        train_f1 = self.train_f1.compute()
        # log metrics
        self.log("epoch_train_accuracy", train_accuracy)
        self.log("epoch_train_f1", train_f1)
        # reset all metrics
        self.train_acc.reset()
        self.train_f1.reset()
        print(f"\ntraining accuracy: {train_accuracy:.4}, "\
        f"f1: {train_f1:.4}")
        
    def validation_epoch_end(self, outputs):
        # outputs = list of dictionaries
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        val_accuracy = self.val_acc.compute()
        val_f1 = self.val_f1.compute()
        # log metrics
        self.log("val_accuracy", val_accuracy)
        self.log("val_loss", avg_loss)
        self.log("val_f1", val_f1)
        # reset all metrics
        self.val_acc.reset()
        self.val_f1.reset()
        print(f"\nvalidation accuracy: {val_accuracy:.4} "\
        f"f1: {val_f1:.4}")
        
        tensorboard_logs = {'avg_val_loss': avg_loss}
        # use key 'log'
        return {'val_loss': avg_loss, 'log': tensorboard_logs}
    
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())


In [36]:
# seeding for torch, numpy, stdlib random, including DataLoader workers!
#seed_everything(123, workers=True)

early_stopping = EarlyStopping(
    monitor="val_loss",
    stopping_threshold=1e-6,
    divergence_threshold=9.0,
    check_finite=True)

In [37]:
model = LitLSTM(dimension=X_train_head_reshaped.shape[2],hidd_dim=256,num_classes=3)
trainer = Trainer(accelerator='gpu',devices=1,max_epochs=200)#,callbacks=[early_stopping])
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params
-------------------------------------------------
0 | mha       | MultiheadAttention | 263 K 
1 | drop      | Dropout            | 0     
2 | fc        | Linear             | 771   
3 | act       | Softmax            | 0     
4 | LSTM      | LSTM               | 3.2 M 
5 | model     | Sequential         | 33.3 K
6 | train_acc | Accuracy           | 0     
7 | train_f1  | F1Score            | 0     
8 | val_acc   | Accuracy           | 0     
9 | val_f1    | F1Score            | 0     
-------------------

Sanity Checking: 0it [00:00, ?it/s]


validation accuracy: 0.3232 f1: 0.3232


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...


validation accuracy: 0.4947 f1: 0.4947

training accuracy: 0.6212, f1: 0.6212


Validation: 0it [00:00, ?it/s]


validation accuracy: 0.4905 f1: 0.4905

training accuracy: 0.6129, f1: 0.6129


Validation: 0it [00:00, ?it/s]


validation accuracy: 0.5173 f1: 0.5173

training accuracy: 0.61, f1: 0.61
